In [1]:
import csv

def compare_definitions(filename):
    """
    Compares definitions in a TSV file and counts matches.

    Args:
        filename (str): The path to the TSV file.
    """
    exact_match = 0
    ignore_case_match = 0
    ignore_period_match = 0
    ignore_case_period_match = 0
    total_rows = 0

    with open(filename, 'r', encoding='utf-8') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        header = next(reader)
        total_rows = 0

        # Determine the correct column for the model's output
        if 'model_long_definition' in header:
            model_col_name = 'model_long_definition'
        elif 'model_prediction' in header:
            model_col_name = 'model_prediction'
        else:
            print(f"'{filename}': Could not find 'model_long_definition' or 'model_prediction' column.")
            return

        definition_full_index = header.index('DefinitionFull')
        model_col_index = header.index(model_col_name)

        for row in reader:
            total_rows += 1
            definition_full = row[definition_full_index].strip()
            model_definition = row[model_col_index].strip()

            # 1. Exact match
            if definition_full == model_definition:
                exact_match += 1

            # 2. Ignore first letter case
            if definition_full.lower()[1:] == model_definition.lower()[1:]:
                ignore_case_match += 1

            # 3. Ignore trailing period
            if definition_full.rstrip('.') == model_definition.rstrip('.'):
                ignore_period_match += 1

            # 4. Ignore both case and period
            if definition_full.lower().rstrip('.') == model_definition.lower().rstrip('.'):
                ignore_case_period_match += 1

    print(f"\nResults for: {filename}")
    print(f"Total rows processed: {total_rows}")
    print(f"Exact matches: {exact_match}")
    print(f"Matches ignoring first letter case: {ignore_case_match}")
    print(f"Matches ignoring trailing period: {ignore_period_match}")
    print(f"Matches ignoring both case and period: {ignore_case_period_match}")

# List of files to process
files_to_process = [
    'metrics_detailed_aya-23_few_shot.tsv',
    'metrics_detailed_aya-101_few_shot.tsv',
    'metrics_detailed_Geitje_few_shot.tsv',
    'evaluation_results_per_entry_geitje.tsv',
    'evaluation_results_per_entry_mT5-xl.tsv',
    'evaluation_results_per_entry_aya-101.tsv',
    'evaluation_results_per_entry_AYA_23_SFT.tsv'
]

# Process each file
for file in files_to_process:
    compare_definitions(file)


Results for: metrics_detailed_aya-23_few_shot.tsv
Total rows processed: 3450
Exact matches: 0
Matches ignoring first letter case: 0
Matches ignoring trailing period: 0
Matches ignoring both case and period: 0

Results for: metrics_detailed_aya-101_few_shot.tsv
Total rows processed: 3450
Exact matches: 322
Matches ignoring first letter case: 326
Matches ignoring trailing period: 326
Matches ignoring both case and period: 341

Results for: metrics_detailed_Geitje_few_shot.tsv
Total rows processed: 3450
Exact matches: 2
Matches ignoring first letter case: 2
Matches ignoring trailing period: 5
Matches ignoring both case and period: 7

Results for: evaluation_results_per_entry_geitje.tsv
Total rows processed: 3450
Exact matches: 277
Matches ignoring first letter case: 277
Matches ignoring trailing period: 277
Matches ignoring both case and period: 277

Results for: evaluation_results_per_entry_mT5-xl.tsv
Total rows processed: 3450
Exact matches: 166
Matches ignoring first letter case: 166


In [2]:
import csv
import pandas as pd

def compare_definitions(filename):
    """
    Compares definitions in a TSV file and returns the match counts.

    Args:
        filename (str): The path to the TSV file.

    Returns:
        dict: A dictionary containing the filename and match counts.
    """
    exact_match = 0
    ignore_case_period_match = 0
    total_rows = 0

    with open(filename, 'r', encoding='utf-8') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        header = next(reader)
        
        # Determine the correct column for the model's output
        if 'model_long_definition' in header:
            model_col_name = 'model_long_definition'
        elif 'model_prediction' in header:
            model_col_name = 'model_prediction'
        else:
            print(f"'{filename}': Could not find 'model_long_definition' or 'model_prediction' column.")
            return None

        definition_full_index = header.index('DefinitionFull')
        model_col_index = header.index(model_col_name)

        for row in reader:
            total_rows += 1
            definition_full = row[definition_full_index].strip()
            model_definition = row[model_col_index].strip()

            # 1. Exact match
            if definition_full == model_definition:
                exact_match += 1

            # 2. Ignore both case and period
            if definition_full.lower().rstrip('.') == model_definition.lower().rstrip('.'):
                ignore_case_period_match += 1
    
    return {
        "file": filename,
        "Total Definitions": total_rows,
        "Exact Match": exact_match,
        "Normalized Match (case/period)": ignore_case_period_match
    }

# List of files to process
files_to_process = [
    'metrics_detailed_aya-23_few_shot.tsv',
    'evaluation_results_per_entry_AYA_23_SFT.tsv',
    'metrics_detailed_aya-101_few_shot.tsv',
    'evaluation_results_per_entry_aya-101.tsv',
    'metrics_detailed_Geitje_few_shot.tsv',
    'evaluation_results_per_entry_geitje.tsv',
    'evaluation_results_per_entry_mT5-xl.tsv'
]

# A more descriptive naming scheme for the models
model_name_map = {
    'metrics_detailed_aya-23_few_shot.tsv': 'AYA-23 (Few-shot)',
    'evaluation_results_per_entry_AYA_23_SFT.tsv': 'AYA-23 (Fine-tuned)',
    'metrics_detailed_aya-101_few_shot.tsv': 'AYA-101 (Few-shot)',
    'evaluation_results_per_entry_aya-101.tsv': 'AYA-101 (Fine-tuned)',
    'metrics_detailed_Geitje_few_shot.tsv': 'Geitje (Few-shot)',
    'evaluation_results_per_entry_geitje.tsv': 'Geitje (Fine-tuned)',
    'evaluation_results_per_entry_mT5-xl.tsv': 'mT5-xl (Fine-tuned)'
}

# Process each file and store the results
results_list = []
for file in files_to_process:
    result = compare_definitions(file)
    if result:
        results_list.append(result)

# Create a pandas DataFrame for a nice table
df = pd.DataFrame(results_list)

# Use the descriptive names
df['Model'] = df['file'].map(model_name_map)
df = df.set_index('Model')
df = df.drop(columns=['file'])

# Calculate percentages
df['Exact Match (%)'] = (df['Exact Match'] / df['Total Definitions'] * 100).round(2)
df['Normalized Match (%)'] = (df['Normalized Match (case/period)'] / df['Total Definitions'] * 100).round(2)

# Reorder columns for clarity
df = df[[
    'Total Definitions', 
    'Exact Match', 
    'Exact Match (%)', 
    'Normalized Match (case/period)', 
    'Normalized Match (%)'
]]


# Save the table to a CSV file
df.to_csv('comparison_results.csv')

print("Results have been saved to 'comparison_results.csv'")
print(df)

Results have been saved to 'comparison_results.csv'
                      Total Definitions  Exact Match  Exact Match (%)  \
Model                                                                   
AYA-23 (Few-shot)                  3450            0             0.00   
AYA-23 (Fine-tuned)                3450          258             7.48   
AYA-101 (Few-shot)                 3450          322             9.33   
AYA-101 (Fine-tuned)               3450          247             7.16   
Geitje (Few-shot)                  3450            2             0.06   
Geitje (Fine-tuned)                3450          277             8.03   
mT5-xl (Fine-tuned)                3450          166             4.81   

                      Normalized Match (case/period)  Normalized Match (%)  
Model                                                                       
AYA-23 (Few-shot)                                  0                  0.00  
AYA-23 (Fine-tuned)                              258       